In [1]:
from google.colab import drive

drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import math

import sys

import numpy as np

import matplotlib.pyplot as plt

import keras

import pandas as pd

import numpy as np

from keras.models import Sequential

from keras.layers import Dense

from keras.layers import LSTM

from keras.layers import Dropout

from keras.layers import *

from sklearn.preprocessing import MinMaxScaler

from sklearn.preprocessing import StandardScaler

from sklearn.metrics import mean_squared_error

from sklearn.metrics import mean_absolute_error

from sklearn.model_selection import train_test_split

from keras.callbacks import EarlyStopping

In [3]:
%cd /content/drive/MyDrive/학교/3학년2학기/소프트웨어융합개론/coin/indicator

/content/drive/MyDrive/학교/3학년2학기/소프트웨어융합개론/coin/indicator


In [4]:
df=pd.read_csv("etherium_indicator.csv")
df_time = pd.read_csv("time.csv")
print('Number of rows and columns:', df.shape)

time_step = 30
index_rsi = 14
n_days = 15

df

Number of rows and columns: (3962, 25)


,close,close_log,nvi,pvi,ma_5,ma_10,ma_20,ma_60,rsi,vpt,...,mfi,ema_5,ema_10,ema_20,ema_60,fi,ubb,mbb,lbb,volume
0,357000.0,12.785491,1.107281,358500.0,358320.0,353255.0,390420.0,3.718542e+05,38.333333,-944127.093549,...,37.597742,3.578846e+05,3.631174e+05,3.782024e+05,3.745695e+05,-1.641898e+10,4.829729e+05,390420.0,2.978671e+05,260618.673054
1,363250.0,12.802847,1.107281,363250.0,366010.0,349980.0,385677.5,3.727342e+05,39.586357,-938285.953078,...,39.471723,3.596731e+05,3.631415e+05,3.767784e+05,3.741984e+05,-1.915128e+10,4.732127e+05,385677.5,2.981423e+05,333645.943705
2,350950.0,12.768399,1.069787,363250.0,363200.0,346720.0,380595.0,3.735300e+05,38.073635,-944992.279851,...,37.899379,3.567654e+05,3.609248e+05,3.743185e+05,3.734362e+05,-1.353704e+10,4.634483e+05,380595.0,2.977417e+05,198054.731732
3,333000.0,12.715898,1.069787,333000.0,352540.0,345915.0,375200.0,3.742308e+05,37.356617,-967092.045407,...,37.080629,3.488436e+05,3.558476e+05,3.703834e+05,3.721104e+05,-3.190942e+10,4.555331e+05,375200.0,2.948669e+05,432084.274204
4,324000.0,12.688499,1.040874,333000.0,345640.0,344625.0,369807.5,3.747342e+05,36.586184,-971914.242266,...,35.739930,3.405624e+05,3.500571e+05,3.659659e+05,3.705330e+05,-1.427370e+10,4.485934e+05,369807.5,2.910216e+05,178421.283776
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3957,1623000.0,14.299787,37.953196,1623000.0,1636000.0,1641200.0,1629050.0,1.847700e+06,61.924686,1629.737036,...,58.453687,1.629896e+06,1.634741e+06,1.649295e+06,1.769474e+06,4.552528e+08,1.700647e+06,1629050.0,1.557453e+06,7986.891172
3958,1652000.0,14.317497,37.953196,1652000.0,1632800.0,1643100.0,1628300.0,1.837800e+06,66.292135,1792.394049,...,64.978170,1.637264e+06,1.637879e+06,1.649552e+06,1.765622e+06,7.919770e+08,1.698524e+06,1628300.0,1.558076e+06,9103.183850
3959,1723000.0,14.359578,37.953196,1723000.0,1645000.0,1653200.0,1630200.0,1.829667e+06,68.641115,2298.189323,...,60.789201,1.665842e+06,1.653355e+06,1.656547e+06,1.764225e+06,1.259245e+09,1.708478e+06,1630200.0,1.551922e+06,11768.644983
3960,1714000.0,14.354340,37.754949,1723000.0,1664800.0,1660700.0,1634400.0,1.821700e+06,66.095890,2262.290537,...,62.129677,1.681895e+06,1.664382e+06,1.662019e+06,1.762578e+06,6.460266e+08,1.721184e+06,1634400.0,1.547616e+06,6872.623278


In [5]:
df_time = df_time.iloc[:,0]

In [6]:
df_time = df_time.str[:10]

In [7]:
res = []

# training set/ test set

In [8]:
feature = df.iloc[:,1:]

idx = int(feature.shape[0]*0.8)

training_set = feature.iloc[:idx].values
test_set = feature.iloc[idx:].values
training_set.shape

(3169, 24)

In [9]:
training_set

array([[1.27854911e+01, 1.10728081e+00, 3.58500000e+05, ...,
        3.90420000e+05, 2.97867122e+05, 2.60618673e+05],
       [1.28028466e+01, 1.10728081e+00, 3.63250000e+05, ...,
        3.85677500e+05, 2.98142283e+05, 3.33645944e+05],
       [1.27683990e+01, 1.06978720e+00, 3.63250000e+05, ...,
        3.80595000e+05, 2.97741662e+05, 1.98054732e+05],
       ...,
       [1.54699218e+01, 1.84396454e+01, 5.23100000e+06, ...,
        5.00580000e+06, 4.77716506e+06, 2.36902652e+04],
       [1.54708774e+01, 1.84572741e+01, 5.23100000e+06, ...,
        5.03100000e+06, 4.81943043e+06, 2.00080298e+04],
       [1.54513956e+01, 1.81011739e+01, 5.23100000e+06, ...,
        5.04185000e+06, 4.83265276e+06, 1.47827157e+04]])

In [10]:
# 정규화
ss = StandardScaler()

training_set_scaled = ss.fit_transform(training_set)

X_train = []

y_train = []

for i in range(time_step, idx-n_days):
    X_train.append(training_set_scaled[i-time_step:i])

    y_train.append(training_set_scaled[i+n_days, 0])

X_train, y_train = np.array(X_train), np.array(y_train)
y_train

array([-0.64991054, -0.6299313 , -0.63149922, ...,  2.37335116,
        2.37429899,  2.35497491])

In [11]:
X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], X_train.shape[2]))
X_train.shape

(3124, 30, 24)

In [12]:
y_train.shape

(3124,)

# 모델 학습

# 모델이 예상한 값 불러오기

In [13]:
dataset_train = feature.iloc[:idx] 

dataset_test = feature.iloc[idx:]

dataset_total = pd.concat((dataset_train, dataset_test), axis = 0)

inputs = dataset_total[len(dataset_total) - len(dataset_test) - time_step:].values

#inputs = inputs.reshape(-1,n_features)

inputs = ss.transform(inputs)

X_test = []
for i in range(time_step, dataset_test.shape[0]):

    X_test.append(inputs[i-time_step:i])

X_test = np.array(X_test)

X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], X_test.shape[2]))

print(X_test.shape)

(763, 30, 24)


In [14]:
def measure_accuarcy(predict, real=dataset_test.values[:,0], n_days=1):
    predict_result = []
    real_result = []
    count = 0
    result = 0
    correct_up = 0
    correct_down = 0
    real_up = 0
    real_down = 0
    
    n_predicting_days = predict.shape[0]-n_days
    
    for i in range(0,n_predicting_days):
        if predict[i] > predict[i+n_days]:
            result = 1
        else:
            result = 0
        predict_result.append(result)
    
    for i in range(0,n_predicting_days):
        if real[i] > real[i+n_days]:
            result = 1
        else:
            result = 0
        real_result.append(result)
        
    for i in range(0,n_predicting_days):
        if real_result[i]==1:
            real_up += 1
            if predict_result[i] ==1:
                correct_up += 1
        elif real_result[i]==0:
            if predict_result[i] == 0:
                correct_down += 1
                
    count = correct_up + correct_down
    
    return (count/n_predicting_days,correct_up,correct_down)

In [15]:
def measure_diff(predict, real = dataset_test.values[:,0], n_days=1):
    
    predict_result = []
    real_result = []
    result = 0
    predict_diff = 0
    
    n_predicting_days = predict.shape[0]-n_days
    
    for i in range(0,n_predicting_days):
        if predict[i] > predict[i+n_days]:
            result = 1
        else:
            result = 0
        predict_result.append(result)
    
    for i in range(0,n_predicting_days):
        if real[i] > real[i+n_days]:
            result = 1
        else:
            result = 0
        real_result.append(result)
        
    for i in range(0,n_predicting_days):
        if real_result[i]==predict_result[i]:
            predict_diff += abs(predict[i]-real[i])
        else:
            predict_diff -= abs(predict[i]-real[i])
    
    return predict_diff

In [16]:
model13 = Sequential()

#Adding the first LSTM layer and some Dropout regularisation

model13.add(LSTM(units = 100, return_sequences = True, input_shape = (X_train.shape[-2:])))

model13.add(LSTM(60))
model13.add(Dropout(0.4))


# Adding the output layer i 
 
model13.add(Dense(units = 1))

# Compiling the RNN

model13.compile(optimizer = 'adam' , loss = 'mean_squared_error' )

# Fitting the RNN to the Training set

model13.fit(X_train, y_train, epochs = 80, batch_size = 32)

Epoch 1/80
98/98 [==============================] - 8s 7ms/step - loss: 0.0886
Epoch 2/80
98/98 [==============================] - 1s 7ms/step - loss: 0.0413
Epoch 3/80
98/98 [==============================] - 1s 7ms/step - loss: 0.0370
Epoch 4/80
98/98 [==============================] - 1s 7ms/step - loss: 0.0364
Epoch 5/80
98/98 [==============================] - 1s 7ms/step - loss: 0.0341
Epoch 6/80
98/98 [==============================] - 1s 7ms/step - loss: 0.0303
Epoch 7/80
98/98 [==============================] - 1s 7ms/step - loss: 0.0308
Epoch 8/80
98/98 [==============================] - 1s 7ms/step - loss: 0.0294
Epoch 9/80
98/98 [==============================] - 1s 7ms/step - loss: 0.0276
Epoch 10/80
98/98 [==============================] - 1s 7ms/step - loss: 0.0268
Epoch 11/80
98/98 [==============================] - 1s 7ms/step - loss: 0.0288
Epoch 12/80
98/98 [==============================] - 1s 7ms/step - loss: 0.0268
Epoch 13/80
98/98 [==============================

In [17]:
predicted_stock_price = model13.predict(X_test)
log = feature.iloc[:feature.shape[0]-X_test.shape[0], 0].values
data_mean = log.mean(axis=0) 
data_std = log.std(axis=0)
original = (predicted_stock_price)*data_std+data_mean

y = feature.iloc[feature.shape[0]-X_test.shape[0]:, 1:2].values
rmse = np.sqrt(np.mean(((original - y) ** 2)))
print(rmse)

res.append(round(measure_accuarcy(original, n_days = 15)[0]  * 100,2))
print(measure_diff(original, n_days = 15))

24/24 [==============================] - 1s 3ms/step
12.155421858772137
[27.680023]


In [18]:
model14 = Sequential()

#Adding the first LSTM layer and some Dropout regularisation

model14.add(LSTM(units = 100, return_sequences = True, input_shape = (X_train.shape[-2:])))

model14.add(LSTM(60))
model14.add(Dropout(0.8))


# Adding the output layer i 
 
model14.add(Dense(units = 1))

# Compiling the RNN

model14.compile(optimizer = 'adam' , loss = 'mean_squared_error' )

# Fitting the RNN to the Training set

model14.fit(X_train, y_train, epochs = 80, batch_size = 32)

Epoch 1/80
98/98 [==============================] - 3s 7ms/step - loss: 0.1801
Epoch 2/80
98/98 [==============================] - 1s 7ms/step - loss: 0.1181
Epoch 3/80
98/98 [==============================] - 1s 7ms/step - loss: 0.1091
Epoch 4/80
98/98 [==============================] - 1s 7ms/step - loss: 0.1022
Epoch 5/80
98/98 [==============================] - 1s 7ms/step - loss: 0.0997
Epoch 6/80
98/98 [==============================] - 1s 7ms/step - loss: 0.0999
Epoch 7/80
98/98 [==============================] - 1s 7ms/step - loss: 0.1005
Epoch 8/80
98/98 [==============================] - 1s 7ms/step - loss: 0.0951
Epoch 9/80
98/98 [==============================] - 1s 7ms/step - loss: 0.0889
Epoch 10/80
98/98 [==============================] - 1s 7ms/step - loss: 0.0919
Epoch 11/80
98/98 [==============================] - 1s 7ms/step - loss: 0.0935
Epoch 12/80
98/98 [==============================] - 1s 7ms/step - loss: 0.0908
Epoch 13/80
98/98 [==============================

In [19]:
predicted_stock_price = model14.predict(X_test)
log = feature.iloc[:feature.shape[0]-X_test.shape[0], 0].values
data_mean = log.mean(axis=0) 
data_std = log.std(axis=0)
original = (predicted_stock_price)*data_std+data_mean

y = feature.iloc[feature.shape[0]-X_test.shape[0]:, 1:2].values
rmse = np.sqrt(np.mean(((original - y) ** 2)))
print(rmse)

res.append(round(measure_accuarcy(original, n_days = 15)[0]  * 100,2))
print(measure_diff(original, n_days = 15))

24/24 [==============================] - 1s 3ms/step
12.234647591269232
[53.92111]


In [20]:
model15 = Sequential()

#Adding the first LSTM layer and some Dropout regularisation

model15.add(LSTM(units = 100, return_sequences = True, input_shape = (X_train.shape[-2:])))

model15.add(LSTM(100))
model15.add(Dropout(0.4))


# Adding the output layer i 
 
model15.add(Dense(units = 1))

# Compiling the RNN

model15.compile(optimizer = 'adam' , loss = 'mean_squared_error' )

# Fitting the RNN to the Training set

model15.fit(X_train, y_train, epochs = 80, batch_size = 32)

Epoch 1/80
98/98 [==============================] - 3s 7ms/step - loss: 0.0743
Epoch 2/80
98/98 [==============================] - 1s 7ms/step - loss: 0.0381
Epoch 3/80
98/98 [==============================] - 1s 7ms/step - loss: 0.0355
Epoch 4/80
98/98 [==============================] - 1s 7ms/step - loss: 0.0307
Epoch 5/80
98/98 [==============================] - 1s 7ms/step - loss: 0.0283
Epoch 6/80
98/98 [==============================] - 1s 7ms/step - loss: 0.0278
Epoch 7/80
98/98 [==============================] - 1s 7ms/step - loss: 0.0259
Epoch 8/80
98/98 [==============================] - 1s 7ms/step - loss: 0.0236
Epoch 9/80
98/98 [==============================] - 1s 7ms/step - loss: 0.0254
Epoch 10/80
98/98 [==============================] - 1s 7ms/step - loss: 0.0222
Epoch 11/80
98/98 [==============================] - 1s 7ms/step - loss: 0.0221
Epoch 12/80
98/98 [==============================] - 1s 7ms/step - loss: 0.0211
Epoch 13/80
98/98 [==============================

In [21]:
predicted_stock_price = model15.predict(X_test)
log = feature.iloc[:feature.shape[0]-X_test.shape[0], 0].values
data_mean = log.mean(axis=0) 
data_std = log.std(axis=0)
original = (predicted_stock_price)*data_std+data_mean

y = feature.iloc[feature.shape[0]-X_test.shape[0]:, 1:2].values
rmse = np.sqrt(np.mean(((original - y) ** 2)))
print(rmse)

res.append(round(measure_accuarcy(original, n_days = 15)[0]  * 100,2))
print(measure_diff(original, n_days = 15))

24/24 [==============================] - 1s 3ms/step
12.151799057151687
[49.9769]


In [22]:
model16 = Sequential()

#Adding the first LSTM layer and some Dropout regularisation

model16.add(LSTM(units = 100, return_sequences = True, input_shape = (X_train.shape[-2:])))

model16.add(LSTM(100))
model16.add(Dropout(0.8))


# Adding the output layer i 
 
model16.add(Dense(units = 1))

# Compiling the RNN

model16.compile(optimizer = 'adam' , loss = 'mean_squared_error' )

# Fitting the RNN to the Training set

model16.fit(X_train, y_train, epochs = 80, batch_size = 32)

Epoch 1/80
98/98 [==============================] - 3s 7ms/step - loss: 0.1631
Epoch 2/80
98/98 [==============================] - 1s 7ms/step - loss: 0.0928
Epoch 3/80
98/98 [==============================] - 1s 7ms/step - loss: 0.0791
Epoch 4/80
98/98 [==============================] - 1s 7ms/step - loss: 0.0817
Epoch 5/80
98/98 [==============================] - 1s 7ms/step - loss: 0.0735
Epoch 6/80
98/98 [==============================] - 1s 7ms/step - loss: 0.0736
Epoch 7/80
98/98 [==============================] - 1s 7ms/step - loss: 0.0728
Epoch 8/80
98/98 [==============================] - 1s 7ms/step - loss: 0.0703
Epoch 9/80
98/98 [==============================] - 1s 7ms/step - loss: 0.0699
Epoch 10/80
98/98 [==============================] - 1s 6ms/step - loss: 0.0653
Epoch 11/80
98/98 [==============================] - 1s 7ms/step - loss: 0.0673
Epoch 12/80
98/98 [==============================] - 1s 7ms/step - loss: 0.0652
Epoch 13/80
98/98 [==============================

In [23]:
predicted_stock_price = model16.predict(X_test)
log = feature.iloc[:feature.shape[0]-X_test.shape[0], 0].values
data_mean = log.mean(axis=0) 
data_std = log.std(axis=0)
original = (predicted_stock_price)*data_std+data_mean

y = feature.iloc[feature.shape[0]-X_test.shape[0]:, 1:2].values
rmse = np.sqrt(np.mean(((original - y) ** 2)))
print(rmse)

res.append(round(measure_accuarcy(original, n_days = 15)[0]  * 100,2))
print(measure_diff(original, n_days = 15))

24/24 [==============================] - 1s 3ms/step
12.22152085140067
[43.66738]


In [24]:
model17 = Sequential()

#Adding the first LSTM layer and some Dropout regularisation

model17.add(LSTM(units = 60, return_sequences = True, input_shape = (X_train.shape[-2:])))

model17.add(LSTM(60))
model17.add(Dropout(0.8))


# Adding the output layer i 
 
model17.add(Dense(units = 1))

# Compiling the RNN

model17.compile(optimizer = 'adam' , loss = 'mean_squared_error' )

# Fitting the RNN to the Training set

model17.fit(X_train, y_train, epochs = 80, batch_size = 32)

Epoch 1/80
98/98 [==============================] - 3s 7ms/step - loss: 0.1970
Epoch 2/80
98/98 [==============================] - 1s 6ms/step - loss: 0.1133
Epoch 3/80
98/98 [==============================] - 1s 7ms/step - loss: 0.1111
Epoch 4/80
98/98 [==============================] - 1s 7ms/step - loss: 0.1001
Epoch 5/80
98/98 [==============================] - 1s 6ms/step - loss: 0.1035
Epoch 6/80
98/98 [==============================] - 1s 6ms/step - loss: 0.0935
Epoch 7/80
98/98 [==============================] - 1s 6ms/step - loss: 0.0919
Epoch 8/80
98/98 [==============================] - 1s 6ms/step - loss: 0.0980
Epoch 9/80
98/98 [==============================] - 1s 6ms/step - loss: 0.0928
Epoch 10/80
98/98 [==============================] - 1s 6ms/step - loss: 0.0922
Epoch 11/80
98/98 [==============================] - 1s 7ms/step - loss: 0.0894
Epoch 12/80
98/98 [==============================] - 1s 7ms/step - loss: 0.0906
Epoch 13/80
98/98 [==============================

In [25]:
predicted_stock_price = model17.predict(X_test)
log = feature.iloc[:feature.shape[0]-X_test.shape[0], 0].values
data_mean = log.mean(axis=0) 
data_std = log.std(axis=0)
original = (predicted_stock_price)*data_std+data_mean

y = feature.iloc[feature.shape[0]-X_test.shape[0]:, 1:2].values
rmse = np.sqrt(np.mean(((original - y) ** 2)))
print(rmse)

res.append(round(measure_accuarcy(original, n_days = 15)[0]  * 100,2))
print(measure_diff(original, n_days = 15))

24/24 [==============================] - 1s 3ms/step
12.143294516241388
[22.546722]


In [26]:
model18 = Sequential()

#Adding the first LSTM layer and some Dropout regularisation

model18.add(LSTM(units = 60, return_sequences = True, input_shape = (X_train.shape[-2:])))

model18.add(LSTM(40))
model18.add(Dropout(0.8))


# Adding the output layer i 
 
model18.add(Dense(units = 1))

# Compiling the RNN

model18.compile(optimizer = 'adam' , loss = 'mean_squared_error' )

# Fitting the RNN to the Training set

model18.fit(X_train, y_train, epochs = 80, batch_size = 32)

Epoch 1/80
98/98 [==============================] - 3s 7ms/step - loss: 0.2639
Epoch 2/80
98/98 [==============================] - 1s 7ms/step - loss: 0.1475
Epoch 3/80
98/98 [==============================] - 1s 7ms/step - loss: 0.1407
Epoch 4/80
98/98 [==============================] - 1s 6ms/step - loss: 0.1398
Epoch 5/80
98/98 [==============================] - 1s 7ms/step - loss: 0.1360
Epoch 6/80
98/98 [==============================] - 1s 6ms/step - loss: 0.1282
Epoch 7/80
98/98 [==============================] - 1s 6ms/step - loss: 0.1193
Epoch 8/80
98/98 [==============================] - 1s 6ms/step - loss: 0.1206
Epoch 9/80
98/98 [==============================] - 1s 7ms/step - loss: 0.1187
Epoch 10/80
98/98 [==============================] - 1s 7ms/step - loss: 0.1127
Epoch 11/80
98/98 [==============================] - 1s 6ms/step - loss: 0.1211
Epoch 12/80
98/98 [==============================] - 1s 6ms/step - loss: 0.1212
Epoch 13/80
98/98 [==============================

In [27]:
predicted_stock_price = model18.predict(X_test)
log = feature.iloc[:feature.shape[0]-X_test.shape[0], 0].values
data_mean = log.mean(axis=0) 
data_std = log.std(axis=0)
original = (predicted_stock_price)*data_std+data_mean

y = feature.iloc[feature.shape[0]-X_test.shape[0]:, 1:2].values
rmse = np.sqrt(np.mean(((original - y) ** 2)))
print(rmse)

res.append(round(measure_accuarcy(original, n_days = 15)[0]  * 100,2))
print(measure_diff(original, n_days = 15))

24/24 [==============================] - 1s 3ms/step
12.305831633665676
[62.474453]


In [28]:
model19 = Sequential()

#Adding the first LSTM layer and some Dropout regularisation

model19.add(LSTM(units = 40, return_sequences = True, input_shape = (X_train.shape[-2:])))

model19.add(LSTM(60))
model19.add(Dropout(0.8))


# Adding the output layer i 
 
model19.add(Dense(units = 1))

# Compiling the RNN

model19.compile(optimizer = 'adam' , loss = 'mean_squared_error' )

# Fitting the RNN to the Training set

model19.fit(X_train, y_train, epochs = 80, batch_size = 32)

Epoch 1/80
98/98 [==============================] - 3s 7ms/step - loss: 0.2101
Epoch 2/80
98/98 [==============================] - 1s 6ms/step - loss: 0.1239
Epoch 3/80
98/98 [==============================] - 1s 7ms/step - loss: 0.1110
Epoch 4/80
98/98 [==============================] - 1s 7ms/step - loss: 0.1049
Epoch 5/80
98/98 [==============================] - 1s 6ms/step - loss: 0.0932
Epoch 6/80
98/98 [==============================] - 1s 6ms/step - loss: 0.0945
Epoch 7/80
98/98 [==============================] - 1s 6ms/step - loss: 0.0956
Epoch 8/80
98/98 [==============================] - 1s 6ms/step - loss: 0.0948
Epoch 9/80
98/98 [==============================] - 1s 6ms/step - loss: 0.1009
Epoch 10/80
98/98 [==============================] - 1s 7ms/step - loss: 0.0945
Epoch 11/80
98/98 [==============================] - 1s 6ms/step - loss: 0.0959
Epoch 12/80
98/98 [==============================] - 1s 6ms/step - loss: 0.0841
Epoch 13/80
98/98 [==============================

In [29]:
predicted_stock_price = model19.predict(X_test)
log = feature.iloc[:feature.shape[0]-X_test.shape[0], 0].values
data_mean = log.mean(axis=0) 
data_std = log.std(axis=0)
original = (predicted_stock_price)*data_std+data_mean

y = feature.iloc[feature.shape[0]-X_test.shape[0]:, 1:2].values
rmse = np.sqrt(np.mean(((original - y) ** 2)))
print(rmse)

res.append(round(measure_accuarcy(original, n_days = 15)[0]  * 100,2))
print(measure_diff(original, n_days = 15))

24/24 [==============================] - 1s 3ms/step
12.238212483326716
[40.81476]


In [30]:
model20 = Sequential()

#Adding the first LSTM layer and some Dropout regularisation

model20.add(LSTM(units = 40, return_sequences = True, input_shape = (X_train.shape[-2:])))

model20.add(LSTM(40))
model20.add(Dropout(0.8))


# Adding the output layer i 
 
model20.add(Dense(units = 1))

# Compiling the RNN

model20.compile(optimizer = 'adam' , loss = 'mean_squared_error' )

# Fitting the RNN to the Training set

model20.fit(X_train, y_train, epochs = 80, batch_size = 32)

Epoch 1/80
98/98 [==============================] - 3s 6ms/step - loss: 0.2556
Epoch 2/80
98/98 [==============================] - 1s 6ms/step - loss: 0.1468
Epoch 3/80
98/98 [==============================] - 1s 6ms/step - loss: 0.1388
Epoch 4/80
98/98 [==============================] - 1s 6ms/step - loss: 0.1372
Epoch 5/80
98/98 [==============================] - 1s 6ms/step - loss: 0.1295
Epoch 6/80
98/98 [==============================] - 1s 6ms/step - loss: 0.1240
Epoch 7/80
98/98 [==============================] - 1s 7ms/step - loss: 0.1174
Epoch 8/80
98/98 [==============================] - 1s 7ms/step - loss: 0.1236
Epoch 9/80
98/98 [==============================] - 1s 6ms/step - loss: 0.1194
Epoch 10/80
98/98 [==============================] - 1s 6ms/step - loss: 0.1220
Epoch 11/80
98/98 [==============================] - 1s 6ms/step - loss: 0.1142
Epoch 12/80
98/98 [==============================] - 1s 6ms/step - loss: 0.1191
Epoch 13/80
98/98 [==============================

In [31]:
predicted_stock_price = model20.predict(X_test)
log = feature.iloc[:feature.shape[0]-X_test.shape[0], 0].values
data_mean = log.mean(axis=0) 
data_std = log.std(axis=0)
original = (predicted_stock_price)*data_std+data_mean

y = feature.iloc[feature.shape[0]-X_test.shape[0]:, 1:2].values
rmse = np.sqrt(np.mean(((original - y) ** 2)))
print(rmse)

res.append(round(measure_accuarcy(original, n_days = 15)[0]  * 100,2))
print(measure_diff(original, n_days = 15))

24/24 [==============================] - 1s 3ms/step
12.312584544307736
[70.98669]


In [32]:
model21 = Sequential()

#Adding the first LSTM layer and some Dropout regularisation

model21.add(LSTM(units = 50, return_sequences = True, input_shape = (X_train.shape[-2:])))

model21.add(LSTM(50))
model21.add(Dropout(0.8))


# Adding the output layer i 
 
model21.add(Dense(units = 1))

# Compiling the RNN

model21.compile(optimizer = 'adam' , loss = 'mean_squared_error' )

# Fitting the RNN to the Training set

model21.fit(X_train, y_train, epochs = 80, batch_size = 32)

Epoch 1/80
98/98 [==============================] - 3s 7ms/step - loss: 0.2428
Epoch 2/80
98/98 [==============================] - 1s 6ms/step - loss: 0.1309
Epoch 3/80
98/98 [==============================] - 1s 6ms/step - loss: 0.1214
Epoch 4/80
98/98 [==============================] - 1s 6ms/step - loss: 0.1181
Epoch 5/80
98/98 [==============================] - 1s 6ms/step - loss: 0.1089
Epoch 6/80
98/98 [==============================] - 1s 7ms/step - loss: 0.1057
Epoch 7/80
98/98 [==============================] - 1s 7ms/step - loss: 0.1097
Epoch 8/80
98/98 [==============================] - 1s 7ms/step - loss: 0.1019
Epoch 9/80
98/98 [==============================] - 1s 7ms/step - loss: 0.1021
Epoch 10/80
98/98 [==============================] - 1s 6ms/step - loss: 0.1032
Epoch 11/80
98/98 [==============================] - 1s 6ms/step - loss: 0.1010
Epoch 12/80
98/98 [==============================] - 1s 7ms/step - loss: 0.1035
Epoch 13/80
98/98 [==============================

In [33]:
predicted_stock_price = model21.predict(X_test)
log = feature.iloc[:feature.shape[0]-X_test.shape[0], 0].values
data_mean = log.mean(axis=0) 
data_std = log.std(axis=0)
original = (predicted_stock_price)*data_std+data_mean

y = feature.iloc[feature.shape[0]-X_test.shape[0]:, 1:2].values
rmse = np.sqrt(np.mean(((original - y) ** 2)))
print(rmse)

res.append(round(measure_accuarcy(original, n_days = 15)[0]  * 100,2))
print(measure_diff(original, n_days = 15))

24/24 [==============================] - 1s 3ms/step
12.218126867663967
[70.711586]


In [34]:
score = pd.DataFrame(res)
score

,0
0,66.98
1,83.02
2,74.73
3,81.82
4,62.70
5,81.42
6,76.47
7,76.20
8,81.82


In [35]:
score.to_csv('res_4.csv', index=False)